In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import GridspecLayout
from tkinter import filedialog
from tkinter import *
from datetime import datetime 
directory=""


from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Code zeigen / verstecken'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)
# import pathlib
# temp = pathlib.PosixPath
# pathlib.PosixPath = pathlib.WindowsPath

def button_clicked(b):
    root = Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory()
    global directory
    directory=folder_selected
def start_processing(b):
    start_writing(text2.value, text1.value, text12.value, text13.value, directory, out)
def change_directory(b):
    root = Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory()
    print("Changed directory to: " + folder_selected)
    global directory
    directory=folder_selected
    
p1=progress=widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
p2=progress=widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
c1=widgets.Valid(description='File uploaded',value=False,)
c2=widgets.Valid(description='File downloaded',value=False,)

b1=widgets.Button(description='Start',disabled=False)
b2=widgets.Button(description='Change Directory', disabled=False)
FORMAT = '%d.%m.%Y'
formatted_date="EXPORT_CON_"+datetime.now().strftime(FORMAT)+".xlsx"
text1=widgets.Text(value="", placeholder="file3.CON", description="Text-File 1")
text12=widgets.Text(value="", placeholder="file2.CON", description="Text-File 2")
text13=widgets.Text(value="", placeholder="file3.CON", description="Text-File 3")
text2=widgets.Text(value=formatted_date, placeholder="EXPORT_CON.xlsx", description="Name Export")

b1.on_click(start_processing)
b2.on_click(change_directory)
grid=GridspecLayout(4,3)
out=widgets.Output(layout={'border':'1px solid black'})
grid[0,0]=text1
grid[1,0]=text12
grid[2,0]=text13
grid[3,0]=text2
grid[0,1]=b1
grid[1,1]=b2
grid[0,2]=out
# grid[0,2]=p1
# grid[1,2]=p2
# grid[0,3]=c1
# grid[1,3]=c2
# grid[]


display(grid)
def start_writing(file_name, textfile_1, textfile_2, textfile_3, directory, out):
    import re
    import xlsxwriter
    import datetime



    filename = directory+"/"+file_name
    text_file1 = directory+"/"+textfile_1
    text_file2 = directory+"/"+textfile_2
    text_file3 = directory+"/"+textfile_3
    patientenbuch_name = ""
    ebmkatalog_name = ""


    dict_num = {
            "3101":"Name des Patienten",
            "3102":"Vorname des Patienten",
            "3103":"Geburtsdatum des Patienten",
            "3110":"Geschlecht des Patienten",
            "3112":"PLZ des Patienten",
            "3113" :"Wohnort",
            "4134":"Versicherung",
            "5000":"Leistungstag",
            "5001":"Gebührennummer",
            "5098":"BSNR",
            "0185099":"Arzt",
    }

    new_dict = {
            "3101":"Name des Patienten",
            "3102":"Vorname des Patienten",
            "3110":"Geschlecht des Patienten",
            "3112":"PLZ des Patienten",
            "5098":"BSNR",
            "0185099":"Arzt",

    }
    dict_index = {
            "3101":0,
            "3102":1,
            "3103":2,
            "3110":3,
            "3112":4,
            "3113":5,
            "4134":6,
            "5000":7,
            "5001":8,
            "5098":9,
            "0185099":10,
            "4121" : 11

    }

    workbook = xlsxwriter.Workbook(filename)
    worksheet = workbook.add_worksheet()

    keyliste = list(dict_num.keys())
    new_keyliste = list(new_dict.keys())


    def remove_num(nr, line):
        return re.sub(".*"+nr, '', line).strip()

    def write_line(key_list, i):
        for j in range(14):
            worksheet.write(i, j, key_list.get(j))

    def conv_date(tag):
        re_day = re.compile("\d{6}")
        if re_day.match(tag):
            date = datetime.datetime.strptime(tag, '%Y%m%d').date()
            return date
        else:
            return ""

    def read_file1():
        with open(text_file1) as f1:
            data=f1.read()
        with open(text_file2) as f2:
            data2=f2.read()
        with open(text_file3) as f3:
            data3=f3.read()
        data+="\n"
        data+=data2
        data+="\n"
        data+=data3
        with open ("combined_file.txt", "w") as f:
            f.write(data)
            f.close()
        with open("combined_file.txt") as f:
            key_list = dict.fromkeys(list(range(10)))
            i = 1
            j = 1
            vorname = ""
            nachname = ""
            tag = ""
            leistung = ""
            wohnort = ""
            date = "None"
            geb = "None"
            versicherung = ""
            gebuhren_nr = ""
            doc = ""
            for line in f:
                re_geb = re.compile("\d\d\d3103")
                re_nr = re.compile("\d\d\d3000")
                re_wohnort = re.compile("\d\d\d3113")
                re_tag = re.compile("\d\d\d5000")
                re_leistung = re.compile("\d\d\d5001")
                re_gebuhr = re.compile("\d\d\d4121")
                re_doc = re.compile("0185099.*")
                re_ver = re.compile("\d\d\d4134")
                re_name = re.compile("\d\d3101")
                re_vorname = re.compile("\d\d3102")

    #             writing the formulas
    #             worksheet.write_formula(f'O{j}', f'=SUBSTITUTE(A{j}, " ", "")&SUBSTITUTE(B{j}, " ", "")')
    #             worksheet.write_formula(f'L{j}', f'=VLOOKUP(O{j}, Patientenbuch!$L$1:$M$5000, 2, FALSE)')
    #             worksheet.write_formula(f'M{j}', f'=VLOOKUP(I{j}, EBM_Katalog!$C$6:$D$3200,2,FALSE)')


                if re_geb.match(line):
                    geb = str(conv_date(remove_num("3103", line)))
                    worksheet.write(j, dict_index.get("3103"), geb)

                if re_doc.match(line):
                    doc = remove_num("0185099", line)
                    worksheet.write(j, dict_index.get("0185099"), doc)

                if re_tag.match(line):
                    tag = remove_num("5000", line)
                    date = str(conv_date(tag))
                    worksheet.write(j, dict_index.get("5000"), date)


                if re_ver.match(line):
                    versicherung = remove_num("4134", line)
                    if versicherung == "":
                        worksheet.write(j, dict_index.get("4134"), "keine Versicherung")
                    else:
                        worksheet.write(j, dict_index.get("4134"), versicherung)

                if re_leistung.match(line):
                    worksheet.write(j, dict_index.get("5001"), remove_num("5001", line))
                    worksheet.write(j, dict_index.get("5000"), date)

                    worksheet.write(j, dict_index.get("3113"), wohnort)
                    worksheet.write(j, dict_index.get("3103"), geb)
                    worksheet.write(j, dict_index.get("4121"), gebuhren_nr)
                    worksheet.write(j, dict_index.get("0185099"), doc)
                    if versicherung == "":
                        worksheet.write(j, dict_index.get("4134"), "keine Versicherung")
                    else:
                        worksheet.write(j, dict_index.get("4134"), versicherung)
                    write_line(key_list, j)
                    j = j + 1

                if re_wohnort.match(line):
                    wohnort = remove_num("3113", line)
                    worksheet.write(i, dict_index.get("3113"), wohnort)

                if re_nr.match(line):
                    key_list.clear()
                    key_list = dict.fromkeys(list(range(12)))
                    i = j

                    vorname = ""
                    nachname = ""
                    tag = ""
                    leistung = ""
                    wohnort = ""
                    date = "None"
                    geb = "None"
                    versicherung = ""
                    gebuhren_nr = ""
                    doc = ""

                else:
                    for key in new_keyliste:
                        reg = re.compile("\d\d\d"+key)
                        if reg.match(line):
                            key_list[dict_index.get(key)] = remove_num(key, line)
                            worksheet.write(i, dict_index.get(key), remove_num(key, line))

    def write_heading():
        j = 0
        for key in keyliste:
            worksheet.write(0, j, dict_num.get(key))
            j = j + 1




#     print("started....")
    with out:
        print("started...")
        
    write_heading()
    read_file1()
    workbook.close()
    with out:
        print("finished writing!")
hide_toggle()

GridspecLayout(children=(Text(value='', description='Text-File 1', layout=Layout(grid_area='widget001'), place…

Changed directory to: C:/Users/kai.follmann
